In [2]:
import os 
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def ReadDataSet(directory_path):
    print("Directory Path:", directory_path)
    files = [file for file in os.listdir(directory_path) if file.split("-")[2] in ["03", "04"]]
    data = pd.concat([pd.read_csv(os.path.join(directory_path, file)).iloc[:, 3:] for file in files], ignore_index=True)
    return data

def NormalizeData(data):
    scaler=StandardScaler()
    NewData=scaler.fit_transform(data)
    return NewData

def SplitData(data):
    x=data.iloc[:, :-1].values
    y=data.iloc[:, -1].values
    return x,y

In [6]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

#Neural Network Model
def NeuralNetworkModel(xTrainShape):
    model = Sequential()
    model.add(Dense(120, input_dim=xTrainShape, activation='relu'))
    model.add(Dense(80, activation='relu'))
    model.add(Dense(80, activation='relu'))
    model.add(Dense(80, activation='relu'))
    model.add(Dense(80, activation='relu'))
    model.add(Dense(80, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

#Fitness Function
def getFitnessFunction(chromosome,xTrain,xTest,yTrain,yTest):
    temp = np.where(chromosome == 1)[0]
    xtrainSelected = xTrain[:, temp]
    xtestSelected = xTest[:, temp]
    model = NeuralNetworkModel(xtrainSelected.shape[1])
    model.fit(xtrainSelected, to_categorical(yTrain), epochs=10, batch_size=32, verbose=0)
    _, fitness = model.evaluate(xtestSelected, to_categorical(yTest), verbose=0)
    return fitness

#Selection Function
def Selection(population,FitnessValue):
    FitnessValue = np.array(FitnessValue)
    probability = FitnessValue / np.sum(FitnessValue)
    return population[np.random.choice(len(population), size=len(population), p=probability)]

#Crossover Function
def Crossover(parent1, parent2):
    if len(parent1):
        temp = np.random.randint(1, len(parent1))
        child1 = np.concatenate((parent1[:temp], parent2[temp:]))
        child2 = np.concatenate((parent2[:temp], parent1[temp:]))
    else:
        child1 = parent2 
        child2 = parent2
    return child1, child2

#Mutation Function with Rate 0.03
def Mutation(chromosome):
    rate=0.03
    for i in range(len(chromosome)):
        if np.random.rand() < rate:
            chromosome[i] =  1 - chromosome[i]
    return chromosome

#Initialize Chromosome Function
def InitializeChromosome(PopulationSize, ChromosomeLength):
    return np.random.randint(2, size=(PopulationSize,ChromosomeLength ))

#GA Function
def GeneticAlgorithm(xTrain,xTest,yTrain,yTest):
    #print("Genetic Algorithm")
    ChromosomeLength=708
    PopulationSize = 25
    Iterations=10
    
    population=InitializeChromosome(PopulationSize,ChromosomeLength)
    #print(population)
    
    for generation in range(Iterations):
        FitnessValue=[getFitnessFunction(chromosome,xTrain,xTest,yTrain,yTest) for chromosome in population]
        #print(f"Fitness Value: {FitnessValue}")
        SelectedPopulation=Selection(population,FitnessValue)
        #print(f"Selected: {SelectedPopulation}")
        NewPopulation=[]
        while len(NewPopulation)<PopulationSize:
            idx = np.random.choice(len(SelectedPopulation), size=2, replace=False)
            parent1, parent2 = SelectedPopulation[idx]
            # print(parent1,parent2)
            offspring1,offspring2=Crossover(parent1.tolist(),parent2.tolist())
            mutated1=Mutation(offspring1)
            mutated2=Mutation(offspring2)
            NewPopulation.extend([mutated1,mutated2])
        population=np.array(NewPopulation)[:PopulationSize]
        BestFitness=max(FitnessValue)
        print(f"Generation {generation+1}, Best Fitness: {BestFitness}")
    BestChromosome=population[np.argmax(FitnessValue)]
    print(f"Best Individual:{BestChromosome}")
    return BestChromosome        


In [ ]:
data = ReadDataSet("/data")
NewData=NormalizeData(data)
x,y=SplitData(data)
xTrain,xTest,yTrain,yTest=train_test_split(x,y,test_size=0.2,random_state=42)
print("X Train shape:", xTrain.shape)
print("X Test shape:", xTest.shape)
print("Y Train shape:", yTrain.shape)
print("Y Test shape:", yTest.shape)

BestIndividual=GeneticAlgorithm(xTrain,xTest,yTrain,yTest)
SelectedFeatures = np.where(BestIndividual == 1)[0]
print("Selected Features/Columns For Happy (Emotion Code : 03) & Sad (Emotion Code : 04):", SelectedFeatures)    
    